In [ ]:
import requests
import pandas as pd
from fredapi import Fred

fred = Fred(api_key = 'Your API key')
api_key = ('Your API key')

start_date = '1970-01-01'
end_date = '2025-04-23'

In [ ]:
# Monetary Policy - Effective Federal Funds Rate
series_id_FEDFUNDS = 'FEDFUNDS'
units_FEDFUNDS = 'lin'        # No transformation (level data)
frequency_FEDFUNDS = 'm'      # Monthly 


# URL and parameters
url_FEDFUNDS = "https://api.stlouisfed.org/fred/series/observations"
params_FEDFUNDS = {
    'series_id': series_id_FEDFUNDS,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_FEDFUNDS,
    'frequency': frequency_FEDFUNDS,
    'observation_start': start_date,
    'observation_end': end_date
}

# Make the request
response_FEDFUNDS = requests.get(url_FEDFUNDS, params=params_FEDFUNDS)

# Format response
if response_FEDFUNDS.status_code == 200:
    data_FEDFUNDS = response_FEDFUNDS.json()
    df_FEDFUNDS = pd.DataFrame(data_FEDFUNDS['observations'])

    df_FEDFUNDS['date'] = pd.to_datetime(df_FEDFUNDS['date'])
    df_FEDFUNDS['value'] = pd.to_numeric(df_FEDFUNDS['value'], errors='coerce')
    df_FEDFUNDS = df_FEDFUNDS[['date', 'value']].set_index('date')
    df_FEDFUNDS = df_FEDFUNDS.dropna()

    df_FEDFUNDS.rename(columns={'value': 'Fed Funds Rate (%)'}, inplace=True)
    df_FEDFUNDS.index.name = "Date"

    print(df_FEDFUNDS.head(10))

else:
    print('Failed to retrieve Fed Funds data.')
    print('Status Code:', response_FEDFUNDS.status_code)
    print(response_FEDFUNDS.text)


            Fed Funds Rate (%)
Date                          
1970-01-01                8.98
1970-02-01                8.98
1970-03-01                7.76
1970-04-01                8.10
1970-05-01                7.95
1970-06-01                7.61
1970-07-01                7.21
1970-08-01                6.62
1970-09-01                6.29
1970-10-01                6.20


In [ ]:
# Core CPI - Core Consumer Price Index (YoY % or desired units)
series_id_core_cpi = 'CPILFESL'
units_core_cpi = 'pch'        # Percent change from previous period (you can adjust to 'pc1' for YoY if needed)
frequency_core_cpi = 'm'      # Monthly 
start_date_core_cpi = '1970-01-01'
end_date_core_cpi = '2025-04-23'

# URL and parameters
url_core_cpi = "https://api.stlouisfed.org/fred/series/observations"
params_core_cpi = {
    'series_id': series_id_core_cpi,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_core_cpi,
    'frequency': frequency_core_cpi,
    'observation_start': start_date,
    'observation_end': end_date
}

# Make the request
response_core_cpi = requests.get(url_core_cpi, params=params_core_cpi)

# Format response
if response_core_cpi.status_code == 200:
    data_core_cpi = response_core_cpi.json()
    df_core_cpi = pd.DataFrame(data_core_cpi['observations'])

    df_core_cpi['date'] = pd.to_datetime(df_core_cpi['date'])
    df_core_cpi['value'] = pd.to_numeric(df_core_cpi['value'], errors='coerce')
    df_core_cpi = df_core_cpi[['date', 'value']].set_index('date')
    df_core_cpi = df_core_cpi.dropna()

    df_core_cpi.rename(columns={'value': 'Core CPI (MoM %)'}, inplace=True)
    df_core_cpi.index.name = "Date"

    print(df_core_cpi.head(10))

else:
    print('Failed to retrieve Core CPI data.')
    print('Status Code:', response_core_cpi.status_code)
    print(response_core_cpi.text)


            Core CPI (MoM %)
Date                        
1970-01-01           0.50761
1970-02-01           0.50505
1970-03-01           0.75377
1970-04-01           0.74813
1970-05-01           0.24752
1970-06-01           0.74074
1970-07-01           0.24510
1970-08-01           0.48900
1970-09-01           0.48662
1970-10-01           0.48426


In [10]:
# Merge the two dataframes on the Date index
df_real_rate = df_FEDFUNDS.merge(df_core_cpi, how='inner', left_index=True, right_index=True)

# Subtract Core CPI from Fed Funds to compute the real rate
df_real_rate['Real Fed Funds Rate (%)'] = df_real_rate['Fed Funds Rate (%)'] - df_real_rate['Core CPI (MoM %)']

# Optional: round for readability
df_real_rate = df_real_rate.round(2)

# Preview result
print(df_real_rate.head(10))


            Fed Funds Rate (%)  Core CPI (MoM %)  Real Fed Funds Rate (%)
Date                                                                     
1970-01-01                8.98              0.51                     8.47
1970-02-01                8.98              0.51                     8.47
1970-03-01                7.76              0.75                     7.01
1970-04-01                8.10              0.75                     7.35
1970-05-01                7.95              0.25                     7.70
1970-06-01                7.61              0.74                     6.87
1970-07-01                7.21              0.25                     6.96
1970-08-01                6.62              0.49                     6.13
1970-09-01                6.29              0.49                     5.80
1970-10-01                6.20              0.48                     5.72


In [ ]:
# Policy - Fed Balance Sheet (Total Assets)
series_id_WALCL = 'WALCL'
units_WALCL = 'pch'        # Percent change (Month-over-Month %)
frequency_WALCL = 'm'      # Monthly 


# URL and parameters
url_WALCL = "https://api.stlouisfed.org/fred/series/observations"
params_WALCL = {
    'series_id': series_id_WALCL,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_WALCL,
    'frequency': frequency_WALCL,
    'observation_start': start_date,
    'observation_end': end_date


# Make the request
response_WALCL = requests.get(url_WALCL, params=params_WALCL)

# Format response
if response_WALCL.status_code == 200:
    data_WALCL = response_WALCL.json()
    df_WALCL = pd.DataFrame(data_WALCL['observations'])

    df_WALCL['date'] = pd.to_datetime(df_WALCL['date'])
    df_WALCL['value'] = pd.to_numeric(df_WALCL['value'], errors='coerce')
    df_WALCL = df_WALCL[['date', 'value']].set_index('date')
    df_WALCL = df_WALCL.dropna()

    df_WALCL.rename(columns={'value': 'Fed Balance Sheet MoM %'}, inplace=True)
    df_WALCL.index.name = "Date"

    print(df_WALCL.head(10))

else:
    print('Failed to retrieve WALCL data.')
    print('Status Code:', response_WALCL.status_code)
    print(response_WALCL.text)


            Fed Balance Sheet MoM %
Date                               
2003-02-01                 -0.46810
2003-03-01                  0.32236
2003-04-01                  1.83447
2003-05-01                  0.19427
2003-06-01                  0.04791
2003-07-01                  0.39859
2003-08-01                 -0.19412
2003-09-01                  0.40397
2003-10-01                  0.51959
2003-11-01                  0.09810


In [ ]:
# Fiscal Policy - Federal Budget Deficit as % of GDP
series_id_fiscal = 'FYFSGDA188S'
units_fiscal = 'lin'        # Level data (not percent change)
frequency_fiscal = 'a'      # Quarterly


# URL and parameters
url_fiscal = "https://api.stlouisfed.org/fred/series/observations"
params_fiscal = {
    'series_id': series_id_fiscal,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_fiscal,
    'frequency': frequency_fiscal,
    'observation_start': start_date,
    'observation_end': end_date
}

# Make the request
response_fiscal = requests.get(url_fiscal, params=params_fiscal)

# Format response
if response_fiscal.status_code == 200:
    data_fiscal = response_fiscal.json()
    df_fiscal = pd.DataFrame(data_fiscal['observations'])

    df_fiscal['date'] = pd.to_datetime(df_fiscal['date'])
    df_fiscal['value'] = pd.to_numeric(df_fiscal['value'], errors='coerce')
    df_fiscal = df_fiscal[['date', 'value']].set_index('date')
    df_fiscal = df_fiscal.dropna()

    df_fiscal.rename(columns={'value': 'Federal Budget Deficit (% GDP)'}, inplace=True)
    df_fiscal.index.name = "Date"

    print(df_fiscal.head(10))

else:
    print('Failed to retrieve fiscal data.')
    print('Status Code:', response_fiscal.status_code)
    print(response_fiscal.text)


            Federal Budget Deficit (% GDP)
Date                                      
1970-01-01                        -0.26479
1971-01-01                        -1.97734
1972-01-01                        -1.82729
1973-01-01                        -1.04590
1974-01-01                        -0.39702
1975-01-01                        -3.15994
1976-01-01                        -3.93571
1977-01-01                        -2.57750
1978-01-01                        -2.51680
1979-01-01                        -1.55009


In [19]:
# ---- STEP 1: Concatenate all Growth Theme indicators ----
df_policy_data = pd.concat([
    df_fiscal,
    df_WALCL,
    df_real_rate,
    df_FEDFUNDS,
], axis=1)

# ---- STEP 2: Drop rows with missing values (e.g., incomplete early dates) ----
first_valid_date = df_policy_data.dropna().index.min()
df_policy_data = df_policy_data.loc[first_valid_date:]

# ---- STEP 3: Forward-fill quarterly values like GDP if needed ----
df_policy_data = df_policy_data.ffill()

# ---- STEP 4: Round only numeric columns safely ----
numeric_cols = df_policy_data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df_policy_data[col] = df_policy_data[col].round(2)

# ---- STEP 5: Reset index and name it 'Date' for clean presentation ----
df_policy_data = df_policy_data.reset_index()
df_policy_data.rename(columns={'index': 'Date'}, inplace=True)

# ---- STEP 6: Preview final cleaned dataset ----
print(df_policy_data.head())

        Date  Federal Budget Deficit (% GDP)  Fed Balance Sheet MoM %  \
0 2004-01-01                           -3.38                     0.02   
1 2004-02-01                           -3.38                    -0.83   
2 2004-03-01                           -3.38                     0.25   
3 2004-04-01                           -3.38                     0.88   
4 2004-05-01                           -3.38                     0.10   

   Fed Funds Rate (%)  Core CPI (MoM %)  Real Fed Funds Rate (%)  \
0                1.00              0.21                     0.79   
1                1.01              0.15                     0.86   
2                1.00              0.31                     0.69   
3                1.00              0.20                     0.80   
4                1.00              0.15                     0.85   

   Fed Funds Rate (%)  
0                1.00  
1                1.01  
2                1.00  
3                1.00  
4                1.00  


In [27]:
df_policy_data.to_csv("policy_data.csv", index = False)
